In [ ]:
#An example of the spectrogram visualization
import matplotlib.pyplot as plt
import librosa.display
i
fname = 'data/test/ Otus_angelinae/Otusangelinae141909-extract3.wav'
y, sr = librosa.load(fname)
S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)

plt.figure(figsize=(20, 5))
S_dB = librosa.power_to_db(S, ref=np.max)
librosa.display.specshow(S_dB, x_axis='time',
                          y_axis='mel', sr=sr)
#plt.colorbar(format='%+2.0f dB')
plt.title('Mel-frequency spectrogram')
#plt.tight_layout()
plt.show()